In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
# Calling api key from .env
load_dotenv()

True

In [2]:
apikey = os.getenv('OPENAI_API_KEY')
llm = OpenAI(temperature=0.7)

In [3]:
#Database info
db_user = "test"
db_password = os.getenv("db_password")
db_host = "localhost"
db_name="atliq_tshirts"

In [4]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [12]:
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)
ans1 = db_chain.run("How many t-shirts do we have left for levi in extra large size and red colour?")

/home/computervision/Desktop/siddhant/research/SQL Query Extractor using Langchain/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for levi in extra large size and red colour?
SQLQuery:SELECT COUNT(*) FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'
SQLResult: [(1,)]
Answer:1
> Finished chain.


Wrong answer. So we have to provide query and store answer to use later


In [20]:
qns1 = db_chain.run("SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'
SQLQuery:SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'
SQLResult: [(Decimal('41'),)]
Answer:There are 41 Levi XL Red t-shirts in stock.
> Finished chain.


In [14]:
qns2 = db_chain.run("how much si the price of the inventory of all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
how much si the price of the inventory of all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) AS `total_price` FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17277'),)]
Answer:The total price of the inventory of all small size t-shirts is $17277.
> Finished chain.


Right answer but let me get the answer through query only!!

In [15]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) AS `total_price` FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) AS `total_price` FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) AS `total_price` FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17277'),)]
Answer:The total price for all t-shirts with size 'S' is $17277.
> Finished chain.


In [16]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")




> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(discounts.pct_discount * t_shirts.price/100) AS revenue
FROM discounts
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE t_shirts.brand = 'Levi' AND discounts.pct_discount > 0 AND t_shirts.stock_quantity > 0 AND discounts.pct_discount <= 100 AND t_shirts.price >= 10 AND t_shirts.price <= 50
LIMIT 5;
SQLResult: [(Decimal('17.150000'),)]
Answer:The store will generate $17.15 in revenue after discounts.
> Finished chain.


In [17]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue FROM (SELECT sum(price * stock_quantity) as total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('30202.950000'),)]
Answer:$30,202.95
> Finished chain.


Lets add some more queries


In [18]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('31316'),)]
Answer:The total revenue for all Levi t-shirts in stock is $313.16.
> Finished chain.


In [19]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('398'),)]
Answer:398
> Finished chain.


**Few shot learning**

We will use few shot learning to fix issues we have seen so far

In [21]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for levi in extra large size and red colour?",
     'SQLQuery' : "SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/home/computervision/Desktop/siddhant/research/SQL Query Extractor using Langchain/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/computervision/Desktop/siddhant/research/SQL Query Extractor using Langchain/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/computervision/Desktop/siddhant/research/SQL Query Extractor using Langchain/venv/lib/python3.10/site-pac

In [24]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for levi in extra large size and red colour? SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red' Result of the SQL query There are 41 Levi XL Red t-shirts in stock.",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query The total price for all t-shirts with size 'S' is $17277.",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query $30,202.95",
 "If we have to sell all the Levi’s T-shirts today. How much r

In [25]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas= few_shots)

In [26]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore, k=2)

In [27]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': 'There are 41 Levi XL Red t-shirts in stock.',
  'Question': 'How many t-shirts do we have left for levi in extra large size and red colour?',
  'SQLQuery': "SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Levi' AND size = 'XL' AND color = 'Red'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'The total revenue for all Levi t-shirts in stock is $313.16.',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [28]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [29]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [30]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [31]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(example_selector=example_selector,
                      example_prompt=example_prompt,
                      prefix=_mysql_prompt,
                      suffix=PROMPT_SUFFIX,
                      input_variables=["input", "table_info", "top_k"])

In [36]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
new_chain.run("How many t-shirts do we have left for nike in extra small size and black colour?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and black colour?
SQLQuery:SELECT SUM(stock_quantity) AS total_tshirts FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'Black'
SQLResult: [(Decimal('97'),)]
Answer:There are 97 Nike XS Black t-shirts in stock.
> Finished chain.


'There are 97 Nike XS Black t-shirts in stock.'

In [37]:
new_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * stock_quantity * (1 - pct_discount/100)) FROM t_shirts LEFT JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi'
SQLResult: [(Decimal('5496.950000'),)]
Answer:The total revenue for all Levi t-shirts in stock, with discounts applied, is $5,496.95.
> Finished chain.


'The total revenue for all Levi t-shirts in stock, with discounts applied, is $5,496.95.'